In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os

from torchinfo import summary

# move the model to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [5]:
model_ft = models.convnext_base(pretrained=True)  # or _base, _small, _large
print(model_ft)

Downloading: "https://download.pytorch.org/models/convnext_base-6075fbad.pth" to C:\Users\KimieCrafter/.cache\torch\hub\checkpoints\convnext_base-6075fbad.pth
100%|██████████| 338M/338M [01:17<00:00, 4.59MB/s] 


ConvNeXt(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      (1): LayerNorm2d((128,), eps=1e-06, elementwise_affine=True)
    )
    (1): Sequential(
      (0): CNBlock(
        (block): Sequential(
          (0): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
          (1): Permute()
          (2): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
          (3): Linear(in_features=128, out_features=512, bias=True)
          (4): GELU(approximate='none')
          (5): Linear(in_features=512, out_features=128, bias=True)
          (6): Permute()
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): CNBlock(
        (block): Sequential(
          (0): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
          (1): Permute()
          (2): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
          (3): Linear(

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_ft = model_ft.to(device)


In [7]:
summary(model=model_ft,
        input_size=(1, 3, 224, 224), # make sure this is "input_size", not "input_shape"
        # col_names=["input_size"], # uncomment for smaller output
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

Layer (type (var_name))                                 Input Shape          Output Shape         Param #              Trainable
ConvNeXt (ConvNeXt)                                     [1, 3, 224, 224]     [1, 1000]            --                   True
├─Sequential (features)                                 [1, 3, 224, 224]     [1, 1024, 7, 7]      --                   True
│    └─Conv2dNormActivation (0)                         [1, 3, 224, 224]     [1, 128, 56, 56]     --                   True
│    │    └─Conv2d (0)                                  [1, 3, 224, 224]     [1, 128, 56, 56]     6,272                True
│    │    └─LayerNorm2d (1)                             [1, 128, 56, 56]     [1, 128, 56, 56]     256                  True
│    └─Sequential (1)                                   [1, 128, 56, 56]     [1, 128, 56, 56]     --                   True
│    │    └─CNBlock (0)                                 [1, 128, 56, 56]     [1, 128, 56, 56]     138,496              True
│  

In [9]:
num_ftrs = model_ft.classifier[2].in_features  # Access the in_features of the last Linear layer in the classifier
# Here the size of each output sample is set to 10.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(train_data.classes)).
model_ft.classifier[2] = nn.Linear(num_ftrs, 10)

# LOSS AND OPTIMIZER
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# move the model to GPU
model_ft = model_ft.to(device)

In [ ]:
num_epochs = 10
trained_model, history = train_and_validate(model_ft, loss_fn, optimizer, train_dataloader, test_dataloader, num_epochs)